In [ ]:
import numpy as np
import pandas as pd
import pylab as pl
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport measure_fairness

import measure_fairness as fs

In [ ]:
from os import environ
data_dir = environ.get('data_dir', 'history/2015-03-18/min_share_tolerance/data/')
target = environ.get('target', 'operations')

In [ ]:
print("Data dir: {}".format(data_dir))
print("Target: {}".format(target))

In [ ]:
usage, fair_share, demand = fs.load_data(data_dir, target)

In [ ]:
T = 7500

pools_usage = fs.get_pools_aggregation(usage[:T])
pools_fair_share = fs.get_pools_aggregation(fair_share[:T])
pools_delay = fs.get_pools_aggregated_delay(usage[:T], fair_share[:T])

pools_window_delay_200 = fs.get_pools_aggregated_window_delay(usage[:T], fair_share[:T], 200)
pools_window_delay_2000 = fs.get_pools_aggregated_window_delay(usage[:T], fair_share[:T], 2000)
pools_window_demand_200 = fs.get_pools_window_aggregation(demand[:T], 200)
pools_window_demand_2000 = fs.get_pools_window_aggregation(demand[:T], 2000)

In [ ]:
plt.plot(pools_usage, 'b')
plt.plot(pools_fair_share, 'r')
plt.show()

print("Usage: {}".format(np.mean(pools_usage)))
print("Fair Share: {}".format(np.mean(pools_fair_share)))

In [ ]:
plt.plot(pools_delay)
plt.show()
print("Mean delay: {}".format(np.mean(pools_delay)))

In [ ]:
print("Window size: {}".format(200))
print("Mean delay: {}".format(np.mean(pools_window_delay_200)))
plt.plot(pools_window_delay_200)
plt.show()
plt.plot(pools_window_demand_200)
plt.show()

In [ ]:
print("Window size: {}".format(2000))
print("Mean delay: {}".format(np.mean(pools_window_delay_2000)))
plt.plot(pools_window_delay_2000)
plt.show()
plt.plot(pools_window_demand_2000)
plt.show()

In [ ]:
aggregated_usage = fs.aggregate(usage[:T])
print("Aggregated usage: {}".format(sum(aggregated_usage.values())))

aggregated_fair_share = fs.aggregate(fair_share[:T])
print("Aggregated fair_share: {}".format(sum(aggregated_fair_share.values())))

zero_fs_pools = set()
for pool in aggregated_fair_share:
    if abs(aggregated_fair_share[pool]) < 1e-9:
        zero_fs_pools.add(pool)
        
aggregated_satisfaction = {pool:(aggregated_usage[pool] + 1e-9) / (aggregated_fair_share[pool] + 1e-9) 
                           for pool in aggregated_fair_share.keys() if pool not in zero_fs_pools}
print("Aggregated satisfaction: {}".format(sum(aggregated_satisfaction.values())))

In [ ]:
fs.plot_satisfaction_histogram(aggregated_satisfaction, aggregated_usage)

In [ ]:
sorted_satisfaction = sorted(aggregated_satisfaction.items(), key = lambda x: x[1], reverse=True)

satisfied = sorted_satisfaction[:10]
underserved = sorted_satisfaction[-10:]

print("\nSatisfied:")
for name, value in satisfied:
    usage_val = aggregated_usage[name]
    fair_share_val = aggregated_fair_share[name]
    print("{:10}: {}, usage: {}, fair_share: {}".format(name, value, usage_val, fair_share_val))

print("\nUnderserved:")
for name, value in underserved:
    usage_val = aggregated_usage[name]
    fair_share_val = aggregated_fair_share[name]
    print("{:10}: {}, usage: {}, fair_share: {}".format(name, value, usage_val, fair_share_val))

In [ ]:
from six import iteritems
for name, usage_val in iteritems(aggregated_usage):
    if name in zero_fs_pools:
        continue
        
    fair_share_val = aggregated_fair_share[name]
    satisfaction_val = aggregated_satisfaction[name]
    if (usage_val > 1000 and satisfaction_val > 1.2):
        print("{:10}: {}, usage: {}, fair_share: {}"
                .format(name, satisfaction_val, usage_val, fair_share_val))

In [ ]:
from six import iteritems
usage_threshold = 0.1

filtered_satisfaction = {name: value for name, value in iteritems(aggregated_satisfaction)
                         if aggregated_usage[name] < usage_threshold}

total_count = len(filtered_satisfaction)
for tolerance in np.linspace(0.1, 1.0, 10):
    count = fs.get_underserved_number(filtered_satisfaction, tolerance)
    print("tolerance: {}, underserved_fraction: {}/{}".format(tolerance, count, total_count))

In [ ]:
underserved_periods = fs.get_pools_underserved_periods(usage[:T], fair_share[:T], 0.9)

In [ ]:
filtered_periods = fs.filter_periods(underserved_periods, 60)
for pool in filtered_periods:
    print("{}: {}".format(pool, filtered_periods[pool]))

In [ ]:
total_underserved_penalty = fs.weighted_penalty_for_underserved(2, underserved_periods)
print("Total underserved penalty: {}".format(total_underserved_penalty))

In [ ]:
total_sum = 0
count = 0
for lst in underserved_periods.values():
    count += len(lst)
    total_sum += sum(lst)
print("Total underserved duration: {}".format(total_sum))